In [2]:
import numpy as np
import pandas as pd
import re
from math import log 

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
#loading dataset
DF= pd.read_csv('inaugural.csv',usecols=['yrprez','speech'])
DF.head(5)

,yrprez,speech
0,1789-Washington,Fellow - Citizens of the Senate and of the Hou...
1,1793-Washington,"Fellow citizens, I am again called upon by the..."
2,1797-Adams,"When it was first perceived, in early times, t..."
3,1801-Jefferson,Friends and Fellow Citizens: Called upon to un...
4,1805-Jefferson,"Proceeding, fellow citizens, to that qualifica..."


In [5]:
print(DF.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   yrprez  56 non-null     object
 1   speech  56 non-null     object
dtypes: object(2)
memory usage: 1.0+ KB
None


#Preprocess


In [6]:
def preprocess(data):
    '''
    takes data as input
    cleans data by 
        -removing all non alphanumeric terms
        -removes common english words 
        -uses lemmatization to find similiar meaning words
    
    returns clean data
    '''
    for i in range(len(data)):
    
        data[i]= re.sub(r'\W|[0-9+]',' ',data[i]) #matches any non-alphanumeric term 
        data[i]= re.sub(r'\s+',' ',data[i]) #matches extra whitespaces

        data[i]= data[i].lower().split() #tokenize the data

    sw= stopwords.words('english') #common english words
    for i in range(len(data)):
        data[i]=[word for word in data[i] if word not in sw] #Removing stop word
        lemma = WordNetLemmatizer()
        data[i]=[lemma.lemmatize(word) for word in data[i]] #Maps word to its base form
    
    return data


In [7]:
data = [i for i in DF['speech']]
data = preprocess(data) #preprocessing the data


#TF IDF TFIDF

In [8]:
def compute_doc_tf(doc):
    '''
    Computes term frequency for each word in a document 
    '''
    doctf={} #stores term frequency of each word
    
    #Counts frequency of each word in doc
    for word in doc:
        if word in doctf:
            doctf[word]+=1
        else:
            doctf[word]=1
    #calculates term frequecy of each word in doc
    for word in doctf:
        doctf[word]= log(1+doctf[word],10)
        
    return doctf 

In [9]:
tf=[] #stores list of dictionary containing term frequencies
for doc in data:
    tf.append(compute_doc_tf(doc))

In [10]:
doc_index={} #stores index of all document containing a word

#Computing index for each word in data
for i,doc in enumerate(data):
    for word in doc:
        try:
            doc_index[word].add(i)
        except:
            doc_index[word] = {i}

In [11]:
idf={} #stores inverse document frequency for each word

#computing freq of word in whole data
for doc in data:
        for word in doc:
            if word in idf:
                idf[word]+=1
            else:
                idf[word]=1

#calulating idf for each word
for word in idf:
    idf[word]=(log(len(data),10) - log(idf[word],10))

In [12]:
tfidf={} #stores the product of term frequency and inverse term frequency

#computing tfidf for each word
for i,doc in enumerate(tf):
    for word in doc:
        tfidf[i,word]= tf[i][word]*idf[word]

In [13]:
print(tf)

[{'fellow': 0.6020599913279623, 'citizen': 0.7781512503836435, 'senate': 0.30102999566398114, 'house': 0.47712125471966244, 'representative': 0.47712125471966244, 'among': 0.30102999566398114, 'vicissitude': 0.30102999566398114, 'incident': 0.30102999566398114, 'life': 0.30102999566398114, 'event': 0.47712125471966244, 'could': 0.6020599913279623, 'filled': 0.30102999566398114, 'greater': 0.30102999566398114, 'anxiety': 0.30102999566398114, 'notification': 0.30102999566398114, 'transmitted': 0.30102999566398114, 'order': 0.47712125471966244, 'received': 0.30102999566398114, 'th': 0.30102999566398114, 'day': 0.47712125471966244, 'present': 0.7781512503836435, 'month': 0.30102999566398114, 'one': 0.6989700043360187, 'hand': 0.6989700043360187, 'summoned': 0.30102999566398114, 'country': 0.7781512503836435, 'whose': 0.47712125471966244, 'voice': 0.47712125471966244, 'never': 0.47712125471966244, 'hear': 0.30102999566398114, 'veneration': 0.30102999566398114, 'love': 0.47712125471966244, '

In [14]:
print(doc_index)

{'fellow': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 34, 36, 39, 40, 41, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55}, 'citizen': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55}, 'senate': {0, 2, 40, 13, 15, 50, 25, 27, 31}, 'house': {0, 32, 2, 36, 43, 13, 50, 27, 31}, 'representative': {0, 33, 2, 3, 34, 7, 8, 9, 11, 12, 13, 49, 21, 22, 25, 27, 28, 31}, 'among': {0, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 17, 18, 19, 20, 22, 23, 24, 25, 26, 27, 28, 29, 30, 33, 34, 35, 37, 40, 41, 42, 44, 45, 46, 47, 48, 49, 51, 54}, 'vicissitude': {0, 5, 9, 10, 12, 24}, 'incident': {0, 33, 7, 14, 16, 25, 26}, 'life': {0, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 16, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 

In [15]:
print(idf)

{'fellow': -0.4220736883887566, 'citizen': -0.7157049619797069, 'senate': 0.5720967679505193, 'house': 0.5177391056279266, 'representative': 0.2710667722865381, 'among': -0.2811957506790088, 'vicissitude': 0.9700367766225569, 'incident': 0.7481880270062005, 'life': -0.5046650039736924, 'event': 0.15712341997970136, 'could': -0.09066106373105476, 'filled': 0.9700367766225569, 'greater': 0.023912157405411527, 'anxiety': 0.5720967679505193, 'notification': 1.7481880270062005, 'transmitted': 1.271066772286538, 'order': -0.08432088570003571, 'received': 0.6690067809585758, 'th': 0.49291552190289467, 'day': -0.29320465815202446, 'present': -0.2060544824331243, 'month': 0.845098040014257, 'one': -0.6532125137753433, 'hand': -0.22493982659349787, 'summoned': 0.9700367766225569, 'country': -0.7781512503836432, 'whose': -0.07135590853566787, 'voice': 0.15712341997970136, 'never': -0.3487219860018558, 'hear': 0.9700367766225569, 'veneration': 1.0492180226701817, 'love': 0.0406178509082642, 'retre

In [16]:
print(tfidf)

{(0, 'fellow'): -0.25411368117109584, (0, 'citizen'): -0.556926711070287, (0, 'senate'): 0.17221828757552246, (0, 'house'): 0.24702433169463217, (0, 'representative'): 0.12933171850616207, (0, 'among'): -0.08464835560763194, (0, 'vicissitude'): 0.29201016666059054, (0, 'incident'): 0.22522703852551912, (0, 'life'): -0.15191930395796366, (0, 'event'): 0.07496692328655959, (0, 'could'): -0.05458339924370267, (0, 'filled'): 0.29201016666059054, (0, 'greater'): 0.007198276640067467, (0, 'anxiety'): 0.17221828757552246, (0, 'notification'): 0.5262570341895003, (0, 'transmitted'): 0.3826292249500471, (0, 'order'): -0.040231286784274276, (0, 'received'): 0.20139110837113405, (0, 'th'): 0.1483823574211374, (0, 'day'): -0.1398941743871436, (0, 'present'): -0.16034155315249018, (0, 'month'): 0.25439985932113074, (0, 'one'): -0.4565759535858934, (0, 'hand'): -0.15722619156940051, (0, 'summoned'): 0.29201016666059054, (0, 'country'): -0.6055193684736276, (0, 'whose'): -0.034045420612199326, (0, 'v

#Ranking

In [17]:
def rank_docs(query):
    '''
    Ranks each document w.r.t query passed as argument 
    Returns a dict containing relevance for each doc
    '''
    query=preprocess(query) #perprocessing query
    score={} #stores ranking score of each doc against query
    for key in tfidf:
        if key[1] in query[0]:
            try:
                score[key[0]]+= tfidf[key]
            except:
                score[key[0]] = tfidf[key]
                
    #sorting score dict in descending order
    score = sorted(score.items(),key= lambda x: x[1] ,reverse = True) 
    return score

In [18]:
query = input() #input query
query = [query] #string query to list query
scores = rank_docs(query) #scoring each doc against query

In the face of our common dangers, in this winter of our hardship, let us remember these timeless words. With hope and virtue, let us brave once more the icy currents, and endure what storms may come


In [19]:
result={} #stores index of all relevant docs with thier ranking score

i= 5 # no. of maximum relevant documents required

#calculating results
for doc in scores:
    if( ( doc[1] <= 0) or  (not i)) :
        break
    result[doc[0]] = doc[1]
    i-=1

In [20]:
#Printing relevance docs info
try:
    m1= min(result.values()) 
    m2= max(result.values())

    for key,value in result.items():
        if value <= 0:
            print("No match found")
            break
        print(DF.iloc[key]['yrprez'],end=' ')
        per= (value - m1)/(m1 + m2) #normalising score
        if per:
            print('%.2f'%(per*100),'%')
        else:
            print('%.2f'%value)
except:
    print('Invalid search sequence')


2009-Obama 1.12
